# ∞-Sector Convergence Demo

This notebook illustrates a toy version of the VID ∞-sector convergence using a finite-dimensional operator.

We use the pseudoinverse of the DLSFH Laplacian, `Delta20_pinv.npy`, to construct a bounded operator
\(K\) with spectral radius strictly less than 1, and then study the convergence of the series
\[ S_N = \sum_{n=0}^N K^n. \]

In an idealized ∞-sector picture, one considers an infinite tower of operators; here we approximate the
∞-sector limit with a large cutoff \(N_{\max}\) and track \(\|S_N - S_{N_{\max}}\|\) as \(N\) increases.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Paths can be adjusted depending on where you run the notebook
L_pinv_path = "../data/Delta20_pinv.npy"  # or "Delta20_pinv.npy" if run from repo root

L_pinv = np.load(L_pinv_path)
print("L^+ shape:", L_pinv.shape)

# Compute operator norm (2-norm) of L^+
op_norm = np.linalg.norm(L_pinv, ord=2)
print("Operator norm ||L^+||_2 =", op_norm)

# Build a bounded operator K with spectral radius < 1
# Here we use K = alpha * (L^+ / ||L^+||_2) with alpha in (0,1)
alpha = 0.7
K = alpha * (L_pinv / op_norm)
print("Approximate operator norm ||K||_2 <=", alpha)


## Constructing the partial sums $S_N$

We define
\[ S_N = I + K + K^2 + \cdots + K^N. \]

For a fixed cutoff \(N_{\max}\), we treat \(S_{N_{\max}}\) as an approximation to the ∞-sector limit and
measure the convergence of \(S_N\) via
\[ \mathrm{err}(N) = \| S_N - S_{N_{\max}} \|_2. \]


In [ ]:
N_max = 50  # proxy for "infinity" in this finite example

dim = K.shape[0]
I = np.eye(dim)

# First, build S_{N_max}
S_Nmax = np.zeros_like(K)
current = I.copy()
for n in range(N_max + 1):
    S_Nmax += current
    current = current @ K

print("Built S_Nmax with N_max =", N_max)

# Now build all partial sums S_N and track errors relative to S_Nmax
errors = []
Ns = list(range(N_max + 1))

S_N = np.zeros_like(K)
current = I.copy()
for N in Ns:
    # add the next term
    if N == 0:
        S_N = current.copy()
    else:
        S_N += current
    # compute error vs S_Nmax
    err = np.linalg.norm(S_N - S_Nmax, ord=2)
    errors.append(err)
    # prepare next power
    current = current @ K

errors = np.array(errors)
print("Initial error (N=0):", errors[0])
print("Final error (N=N_max):", errors[-1])


## Convergence plot

We now plot the error $\mathrm{err}(N) = \|S_N - S_{N_{\max}}\|_2$ as a function of $N$.
A roughly exponential decay in $N$ is a numerical indication of ∞-sector convergence in this toy model.


In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(Ns, errors, marker="o")
ax.set_yscale("log")
ax.set_xlabel("N (partial sum index)")
ax.set_ylabel(r"$\\|S_N - S_{N_{\\max}}\\|_2$")
ax.set_title(r"∞-sector convergence: partial sums of $K^n$")
ax.grid(True, which="both", ls=":", alpha=0.6)

plt.tight_layout()
plt.show()

### Summary

- We built a bounded operator $K$ from the Laplacian pseudoinverse.
- We formed finite partial sums $S_N = \sum_{n=0}^N K^n$.
- For a fixed $N_{\max}$, we observed monotone decay of $\| S_N - S_{N_{\max}} \|_2$.

This toy model numerically demonstrates the type of convergence behavior expected in the VID ∞-sector, where
an infinite tower of discrete geometric operators defines a well-behaved nonperturbative object.
